In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import pandas as pred
import numpy as np
from CONSTANT import *
from tools import *
import torch.nn.functional as F
from torch.utils.data import (
    TensorDataset, DataLoader, SequentialSampler, WeightedRandomSampler)

In [2]:
use_gpu = torch.cuda.is_available()

In [4]:
class DatasetProvider(torch.utils.data.Dataset):
    def __init__(self, target, file_name, train_index, test_index, batch_size):
        self.batch_size = batch_size
        data = pd.read_csv(file_name)
        X, y = join_signals(data, target=target)

        xtrain, ytrain, xtest, ytest = X[train_index], y[train_index], X[test_index], y[test_index]

        self.xtrain = torch.tensor(xtrain, dtype=torch.float32)
        self.xtest = torch.tensor(xtest, dtype=torch.float32)

        self.ytrain = torch.tensor(ytrain, dtype=torch.long)
        self.ytest = torch.tensor(ytest, dtype=torch.long)

        

        print(self.xtrain.shape, self.ytrain.shape, self.xtest.shape, self.ytest.shape)

    def get_data(self):
        train_set = TensorDataset(self.xtrain, self.ytrain)
        test_set = TensorDataset(self.xtest, self.ytest)

        train_loader = DataLoader(train_set,batch_size=self.batch_size, shuffle=True, drop_last=False)
        test_loader = DataLoader(test_set, batch_size=self.batch_size,drop_last=False)

        return train_loader, test_loader

In [5]:
spliter = load_model(
        r'./processed_signal/HKU956/400_4s_step_2s_spliter.pkl')

In [6]:
for k in spliter['loso']:
    train_index = k['train_index']
    test_index = k['test_index']
    break

In [7]:
dataprovider = DatasetProvider(target='valence_label',
                               file_name=r'./processed_signal/HKU956/400_4s_step_2s.csv',
                               train_index=train_index,
                               test_index=test_index,
                               batch_size=256
                            )

torch.Size([18089, 4, 400]) torch.Size([18089, 1]) torch.Size([4638, 4, 400]) torch.Size([4638, 1])


In [8]:
train_loader, test_loader = dataprovider.get_data()

In [9]:
class neuralNetwork(nn.Module):
    def __init__(self, in_dim, layer_num, n_hidden_1, n_hidden_2, out_dim):
        super(neuralNetwork, self).__init__() # super() 函数是用于调用父类(超类)的一个方法
# Sequential()表示将一个有序的模块写在一起，也就相当于将神经网络的层按顺序放在一起，这样可以方便结构显示
        self.layer1 = nn.Sequential(
            nn.Linear(in_dim, n_hidden_1),
            nn.ReLU(True)) # 表示使用ReLU激活函数
        self.layer2 = nn.Sequential(
            nn.Linear(n_hidden_1, n_hidden_2),
            nn.ReLU(True))
        self.layer_m = nn.ModuleList([nn.Linear(n_hidden_2, n_hidden_2) for _ in range(layer_num)])
        self.layer3 = nn.Sequential(
            nn.Linear(n_hidden_2, out_dim),
            nn.ReLU(True))

        # self.sigmoid = nn.Sigmoid()

# 定义向前传播
    def forward(self, x):
        x = x.flatten(start_dim=1)
        x = self.layer1(x)
        x = self.layer2(x)
        for m in self.layer_m:
            x = m(x)
            x = F.relu(x)
        x = self.layer3(x)
        # x = self.sigmoid(x)
        return x

In [10]:
model = neuralNetwork(4*400, 1, 1024, 512, 1)
if use_gpu:
    model = model.cuda()

for param in model.parameters():
    nn.init.normal_(param, mean=0, std=0.01)

In [119]:
ytrain = dataprovider.ytrain
pd.Series(ytrain.reshape(1, -1)[0]).value_counts()

1    9715
0    8374
dtype: int64

In [18]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# criterion = nn.CrossEntropyLoss() # 定义损失函数类型，使用交叉熵
# criterion = nn.BCELoss()
# criterion = nn.NLLLoss()
criterion = nn.BCEWithLogitsLoss()


# optimizer = torch.optim.AdamW(model.parameters(), lr=0.00001) # 定义优化器，使用随机梯度下降
optimizer = torch.optim.AdamW(model.parameters(), lr=0.01) # 定义优化器，使用随机梯度下降
# optimizer = torch.optim.Adadelta(model.parameters(), lr=0.01) # 定义优化器，使用随机梯度下降
# optimizer = torch.optim.Adagrad(model.parameters(), lr=0.01) # 定义优化器，使用随机梯度下降


In [19]:
num_epochs = 5
# 开始模型训练
for epoch in range(num_epochs):
    # print('*' * 10)
    # print(f'epoch {epoch+1}')
    running_loss = 0.0 # 初始值
    running_acc = 0.0
    for i, batch in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        data, label = batch

        # label = label.reshape(1, -1)[0]
        # 使用GPU？
        if use_gpu:
            data = data.cuda()
            label = label.cuda()
        # 向前传播
        out = model(data) # 前向传播
        # print(label)
        loss = criterion(out, label.float()) # 计算loss
        # print(torch.argmax(out, dim=1).float())
        # loss = criterion(torch.argmax(out, dim=1).float(), label.float()) # 计算loss
        # loss = criterion(torch.argmax(out, dim=1), label) # 计算loss
        # loss = F.binary_cross_entropy_with_logits(torch.argmax(out, dim=1).float(), label.float())
        # loss.requires_grad = True
        running_loss += loss.item() # loss求和
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).float().mean()
        # 向后传播
        optimizer.zero_grad() # 梯度归零
        loss.backward() # 后向传播
        optimizer.step() # 更新参数

        if i % 300 == 0:
            print(f'[{epoch+1}/{num_epochs}] Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    print(f'Finish {epoch+1} epoch, Loss: {running_loss/i:.6f}, Acc: {running_acc/i:.6f}')
    
    for name, parms in model.named_parameters():
        if name in ['layer3.0.bias', 'layer3.0.bias']:
            print('\t', name, 'grad_requirs:', parms.requires_grad, 
                  'weight', torch.mean(parms.data),
                  'grad_value:', torch.mean(parms.grad)
                  )

Finish 1 epoch, Loss: 0.693147, Acc: 0.463181
	 layer3.0.bias grad_requirs: True weight tensor(-0.0050, device='cuda:0') grad_value: tensor(0., device='cuda:0')
Finish 2 epoch, Loss: 0.693147, Acc: 0.463125
	 layer3.0.bias grad_requirs: True weight tensor(-0.0050, device='cuda:0') grad_value: tensor(0., device='cuda:0')
Finish 3 epoch, Loss: 0.693147, Acc: 0.462558
	 layer3.0.bias grad_requirs: True weight tensor(-0.0049, device='cuda:0') grad_value: tensor(0., device='cuda:0')
Finish 4 epoch, Loss: 0.693147, Acc: 0.462728
	 layer3.0.bias grad_requirs: True weight tensor(-0.0049, device='cuda:0') grad_value: tensor(0., device='cuda:0')
Finish 5 epoch, Loss: 0.693147, Acc: 0.463096
	 layer3.0.bias grad_requirs: True weight tensor(-0.0049, device='cuda:0') grad_value: tensor(0., device='cuda:0')


In [21]:
for i, batch in enumerate(train_loader, 1): # 枚举函数enumerate返回下标和值
        data, label = batch
        print(data.requires_grad)
        print(data.grad)
        break

False
None


In [22]:
data.shape

torch.Size([64, 4, 400])